In [ ]:
# Install dependencies if needed

!pip install ultralytics # Install image recognition module
!pip install pyserial # Install serial port communication module
!pip install opencv-python # Install another image recognition module
!pip install pyserial # Install module used to communicate with 3D printer
!pip install matplotlib # Install module for making plots

In [ ]:
# Import modules
from ultralytics import YOLO
import time
import serial
import cv2
from matplotlib import pyplot as plt

In [ ]:
model = YOLO("yolov8m.pt") # Load trainded object recognition model

In [ ]:
# Connect 3D printer to the computer using a USB cable
# The ID of the serial port use for this connection can be found by typing "ls /dev/tty*" in the terminal
# By doing this before and after connecting the 3D printer to the computer you can find the port connecting them
# The port address should be something like "/dev/tty.usbserial-14140"
# In a PC the port used can be found by opening "Device Manager" and clicking "Ports"

In [ ]:
# Connect the USB camera to the computer using the correcponding USB cable

In [ ]:
# To send instructions to the printer first stablish the connection
ser=serial.Serial("/dev/tty.usbserial-2120", 115200)
# Now you can send any gcode instruction to directly control the 3d printer from your computer

In [ ]:
# For example to home the motors you can send the intruction "G28"
ser.write(str.encode("G28\r\n"))

In [ ]:
# Move approximately to the middle
ser.write(str.encode("G1 X110 Y80 Z30\r\n"))

In [ ]:
# After connecting a device (e.g., a laser) to an available port 
# in the 3D printer's motherboard (fan, head and bed heating, etc.),
# you can control these devices directly from your computer.

# To send a 0.5 second laser pulse at maximum power 
ser.write(str.encode("M107\r\n")) # Turn off laser
time.sleep(0.1) # Pause
ser.write(str.encode("M106 S250\r\n")) # Turn on laser
time.sleep(0.5) # Pause
ser.write(str.encode("M107\r\n")) # Turn off laser

In [ ]:
# To connect the camera 
camera = cv2.VideoCapture(1) # Connect to USB camera. You may need to change the index to find the desired camera

In [ ]:
# To capture a frame
ret, frame = camera.read() 

In [ ]:
# To display the captured image using Jupyter
plt.imshow(frame) # Show image
plt.title('Your first image')
plt.show()

In [ ]:
# To identify objects in the image
results = model.predict(frame) 

In [ ]:
# This displays the results
 
result=results[0]
org = (50, 50) # Set features of box around objects
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
color = (255, 0, 0)
thickness = 2
for box in result.boxes: # Identify objects in image
  class_id = result.names[box.cls[0].item()] # Name of the object
  cords = box.xyxy[0].tolist()
  cords = [round(x) for x in cords]
  conf = round(box.conf[0].item(), 2) # Confidence in the prediction
  img = cv2.rectangle(frame, (cords[0], cords[1]), (cords[2], cords[3]), (0, 255, 0), 3)
  img=cv2.putText(img,class_id,(cords[0],cords[1]), font, fontScale, color, thickness, cv2.LINE_AA)
  print(cords)
plt.imshow(img) # Show image
plt.show()
time.sleep(5)
camera.release() # Close camera

In [ ]:
# This example homes the 3d printer, moves the camera to a selected postion, and fires the laser

ser=serial.Serial("/dev/tty.usbserial-14140", 115200) # Connect
ser.write(str.encode("M84 X Y Z S12000\r\n")) # Keep motors engaged
ser.write(str.encode("M92 X160 Y160 Z800\r\n")) # Set motor speed
ser.write(str.encode("G28\r\n")) # Home device

ser.write(str.encode("M400\r\n")) # Wait until finish moving
ser.write(str.encode("M118 E1 Iamready\r\n"))  
Tcat = ""
while "Iamready" not in Tcat:
    T = ser.read() 
    Tcat += T.decode()
    Tcat
    
ser.write(str.encode("G1 Z30 X70 Y80\r\n")) # Move to specified position 

ser.write(str.encode("M400\r\n")) # Wait until finish moving
ser.write(str.encode("M118 E1 Iamready\r\n"))  
Tcat = ""
while "Iamready" not in Tcat:
    T = ser.read() 
    Tcat += T.decode()
    Tcat
ser.write(str.encode("M107\r\n")) # Turn off laser
time.sleep(0.1)
ser.write(str.encode("M106 S250\r\n")) # Turn on laser
time.sleep(0.5)
ser.write(str.encode("M107\r\n")) # Turn off laser